In [ ]:
import numpy as np

In [ ]:
def trading_strategy(prices, matrix, initial_liquidity, initial_bitcoins, perc_acquisto, perc_vendita, soglia_vendita=None):
    # Ottieni le dimensioni della matrice
    A, B = matrix.shape
    
    # Inizializzazione della liquidità e dei bitcoin per ogni strategia
    liquidity = initial_liquidity * matrix
    bitcoins = np.zeros((A, B)) + (initial_bitcoins / (A * B))
    
    # Inizializzazione del portafoglio totale
    portfolio_value = []
    
    # Inizializzazione per tracciare lo stato di ogni strategia
    descents_count = np.zeros((A, B), dtype=int)
    ascents_count = np.zeros((A, B), dtype=int)
    last_acquisition_prices = np.zeros((A, B), dtype=float)  # Prezzo medio pesato degli ultimi acquisti
    last_acquisition_spent = np.zeros((A, B), dtype=float)  # Totale speso negli ultimi acquisti
    
    # Flag per sapere se siamo in attesa di salite dopo discese
    waiting_for_ascents = np.zeros((A, B), dtype=bool)
    
    for day in range(1, len(prices)):
        for i in range(A):
            for j in range(B):
                # Gestione delle divergenze
                if prices[day] < prices[day - 1]:  # Prezzo scende
                    if waiting_for_ascents[i, j]:  # Aspettativa non soddisfatta, reset
                        waiting_for_ascents[i, j] = False
                        descents_count[i, j] = 0
                        ascents_count[i, j] = 0
                        
                    descents_count[i, j] += 1  # Incrementa il conteggio delle discese
                    
                elif prices[day] > prices[day - 1]:  # Prezzo sale
                    if not waiting_for_ascents[i, j] and descents_count[i, j] >= i:  # Abbiamo raggiunto il numero di discese richiesto
                        waiting_for_ascents[i, j] = True
                        ascents_count[i, j] = 1  # Inizia a contare le salite
                    elif waiting_for_ascents[i, j]:  # Continuiamo a contare le salite
                        ascents_count[i, j] += 1
                    else:  # Se una salita inattesa avviene prima di completare le discese, reset
                        descents_count[i, j] = 0
                        
                # Esegui l'acquisto se si sono verificate "i" discese consecutive
                if descents_count[i, j] == i and not waiting_for_ascents[i, j]:
                    amount_to_invest = perc_acquisto * liquidity[i, j]
                    bitcoins_bought = amount_to_invest / prices[day]
                    
                    bitcoins[i, j] += bitcoins_bought
                    liquidity[i, j] -= amount_to_invest
                    
                    # Aggiorna il prezzo medio pesato degli ultimi acquisti
                    last_acquisition_spent[i, j] += amount_to_invest
                    last_acquisition_prices[i, j] = ((last_acquisition_prices[i, j] * (last_acquisition_spent[i, j] - amount_to_invest)) + (amount_to_invest * prices[day])) / last_acquisition_spent[i, j]
                
                # Esegui la vendita se si sono verificate "i" discese seguite da "j" salite
                if waiting_for_ascents[i, j] and ascents_count[i, j] == j:
                    if soglia_vendita is None or (prices[day] >= (last_acquisition_prices[i, j] * (1 + soglia_vendita))):
                        bitcoins_to_sell = perc_vendita * bitcoins[i, j]
                        
                        if bitcoins_to_sell > 0:
                            liquidity[i, j] += bitcoins_to_sell * prices[day]
                            bitcoins[i, j] -= bitcoins_to_sell
                            
                            # Resetta il prezzo medio pesato e il totale speso dopo la vendita
                            last_acquisition_prices[i, j] = 0
                            last_acquisition_spent[i, j] = 0
                            
                            # Segnala il successo e forza il reset il giorno successivo
                            waiting_for_ascents[i, j] = False
                            descents_count[i, j] = 0
                            ascents_count[i, j] = 0
        
        # Calcola il valore totale del portafoglio al giorno corrente
        total_portfolio_value = np.sum(liquidity + bitcoins * prices[day])
        portfolio_value.append(total_portfolio_value)
    
    return portfolio_value

# Parametri di esempio
prices = [50000, 49000, 48000, 47000, 46000, 47000, 48000, 49000, 50000, 51000, 52000]  # Esempio di prezzi
matrix = np.array([[0.24, 0.4], [0.1, 0.1], [0.05, 0.11]])
initial_liquidity = 100000  # Liquidità iniziale
initial_bitcoins = 2  # Numero iniziale di bitcoin
perc_acquisto = 0.5  # Percentuale di acquisto
perc_vendita = 0.5  # Percentuale di vendita
soglia_vendita = 0.05  # Soglia di vendita del 5%

# Esecuzione della strategia di trading
portfolio_value = trading_strategy(prices, matrix, initial_liquidity, initial_bitcoins, perc_acquisto, perc_vendita, soglia_vendita)

# Stampa il valore del portafoglio al termine di ogni giorno
print("Valore del portafoglio totale per ogni giorno:")
print(portfolio_value)